In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
from statsmodels.distributions import empirical_distribution
%matplotlib inline

Оценки методом моментов:

1) $N(\mu, \sigma^2):\ \mu\sim\bar{X},\ \sigma^2\sim s^2$

2) $\Gamma(\alpha, \lambda):\ \alpha\sim\frac{\bar{x}}{s^2},\ \lambda\sim\frac{\bar{X^2}}{s^2}$

3) $R(a, b):\ a\sim\bar{X}-\sqrt{3s^2},\ b\sim\bar{X}+\sqrt{3s^2}$

4) $Pois(\lambda):\ \lambda\sim\bar{X}$

5) $Bin(n, p):\ n\sim\frac{\bar{X}^2}{\bar{X}-s^2},\ p\sim1+\bar{X}-\frac{\bar{X^2}}{\bar{X}}$

6) $Geom(p):\ p\sim\frac{1}{\bar{X}}$

7) $Beta(\alpha, \lambda):\ \alpha\sim\frac{(1-\bar{X})(\bar{X}-\bar{X^2})}{s^2},\ \lambda\sim\frac{\bar{X}-\bar{X^2}}{s^2}$


Оценки методом максимального правдоподобия:

1) $N(\mu, \sigma^2):\ \mu\sim\bar{X},\ \sigma^2\sim s^2$

2) $\Gamma(\alpha, \lambda):\ \alpha\sim\frac{\lambda}{\bar{X}}$

3) $R(a, b):\ a\sim\ X_{(1)},\ b\sim X_{(n)}$

4) $Pois(\lambda):\ \lambda\sim\bar{X}$

5) $Bin(n, p):\ p\sim\frac{\bar{X}}{n}$

6) $Geom(p):\ p\sim\frac{1}{\bar{X}}$

7) $Pareto(2, \theta),\ p(x)=\frac{2\theta^2}{x^3}I(x>\theta):\ \theta\sim X_{(1)}$

Функции принимают на вход выборку. Возвращают масив из элементов $\bar{X}, S^{2}, X_{(1)}, X_{(n)}$ для $n\leq sample.size$ соответсвенно.

In [2]:
def X(sample):
    avrg_1 = float(sample[0])

    first = np.array([avrg_1])
    for x in xrange(1,sample.size):
        avrg_1 = (avrg_1*x + sample[x])/(x+1)
            
        first = np.append(first, avrg_1)
        
    return first

def S_2(sample):
    avrg_1 = float(sample[0])
    avrg_2 = float(sample[0])**2

    first = np.array([avrg_1])
    second = np.array([avrg_2])
    for x in xrange(1,sample.size):
        avrg_1 = (avrg_1*x + sample[x])/(x+1)
        avrg_2 = (avrg_2*x + sample[x]**2)/(x+1)
            
        first = np.append(first, avrg_1)
        second = np.append(second, avrg_2)
        
    return second - first**2

def X_first(sample):
    min_el = float(sample[0])

    third = np.array([min_el])
    for x in xrange(1,sample.size):
        if(sample[x] < min_el):
            min_el = sample[x]
            
        third = np.append(third, min_el)
        
    return third

def X_last(sample):
    max_el = sample[0]

    fourth = np.array([max_el])
    for x in xrange(1,sample.size):
        if(sample[x] > max_el):
            max_el = sample[x]
            
        fourth = np.append(fourth, max_el)
        
    return fourth

## $N(\mu, \sigma^2):\ \mu\sim\bar{X},\ \sigma^2\sim s^2\ и\ \mu\sim\bar{X},\ \sigma^2\sim s^2$

### Параметрический брустреп

In [16]:
N = 1000
K = 100
mu = [0, 2.0, 7.0]
sigma = [1, 0.5, 2.0]
size = 3

for x in xrange(0, size):
    # задаем распределение
    norm_rv = sts.norm(loc = mu[x], scale = sigma[x])
    # генерируем выборку размера N
    sample = norm_rv.rvs(N)
    
    # массив оценок параметра
    rating_mu = X(sample)
    rating_sigma = S_2(sample)
    
    # итоговые бутстрепные оценки параметра
    result_mu = np.array([])
    result_sigma = np.array([])
    
    print "При параметрах mu =", mu[x], "sigma =", sigma[x], "выборочная оценка дисперсии оценки параметра:\n"
    
    for y in xrange(0, N):
        # для каждой оценки задаем распределение
        bytstrep_norm_rv = sts.norm(loc = rating_mu[y], scale = rating_sigma[y])
        
        for z in xrange(0, K):
            # генерируем выбороку размера y+1
            bytstrep_sample = bytstrep_norm_rv.rvs(y+1)
            # добавляем итоговые оценки параметра
            result_mu = np.append(result_mu, np.mean(bytstrep_sample) )
            result_sigma = np.append(result_sigma, np.var(bytstrep_sample) )
    
    #выборочная оценка дисперсии оценки параметра
    var_mu =  np.var(result_mu)
    var_sigma = np.var(result_sigma)
    
    print "var_mu =",var_mu, "var_sigma =", var_sigma, '\n'

При параметрах mu = 0 sigma = 1 выборочная оценка дисперсии оценки параметра:

var_mu = 0.0100573563394 var_sigma = 0.034344589736 

При параметрах mu = 2.0 sigma = 0.5 выборочная оценка дисперсии оценки параметра:

var_mu = 0.00528874674893 var_sigma = 0.000268377897333 

При параметрах mu = 7.0 sigma = 2.0 выборочная оценка дисперсии оценки параметра:

var_mu = 0.176950259654 var_sigma = 13.6450031708 



### Непараметрический брустреп

In [18]:
N = 1000
K = 100
mu = [0, 2.0, 7.0]
sigma = [1, 0.5, 3.0]
size = 3

for x in xrange(0, size):
    # задаем распределение
    norm_rv = sts.norm(loc = mu[x], scale = sigma[x])
    # генерируем выборку размера N
    sample = norm_rv.rvs(N)
    
    # итоговые бутстрепные оценки параметра
    result_mu = np.array([])
    result_sigma = np.array([])
    
    print "При параметрах mu =", mu[x], "sigma =", sigma[x], "выборочная оценка дисперсии оценки параметра:\n"
    
    for y in xrange(0, N):
        # для каждой оценки задаем эмпирическое распределение
        bytstrep_ecdf = empirical_distribution.ECDF(sample[:y+1])
        class minus_six_gen+str(y)(sts.rv_continuous):
            def _pdf(self, x):
                return 5./(np.power(x,6))
            minus_six = minus_six_gen(a=1, name='minus_six')
        
        for z in xrange(0, K):
            # генерируем выбороку размера y+1
            bytstrep_sample = bytstrep_ecdf.rvs(y+1)
            # добавляем итоговые оценки параметра
            result_mu = np.append(result_mu, np.mean(bytstrep_sample) )
            result_sigma = np.append(result_sigma, np.var(bytstrep_sample) )
    
    #выборочная оценка дисперсии оценки параметра
    var_mu =  np.var(result_mu)
    var_sigma = np.var(result_sigma)
    
    print "var_mu =",var_mu, "var_sigma =", var_sigma, '\n'

При параметрах mu = 0 sigma = 1 выборочная оценка дисперсии оценки параметра:



AttributeError: 'ECDF' object has no attribute 'rvs'

Оценки методом моментов:

1) $N(\mu, \sigma^2):\ \mu\sim\bar{X},\ \sigma^2\sim s^2$

2) $\Gamma(\alpha, \lambda):\ \alpha\sim\frac{\bar{x}}{s^2},\ \lambda\sim\frac{\bar{X^2}}{s^2}$

3) $R(a, b):\ a\sim\bar{X}-\sqrt{3s^2},\ b\sim\bar{X}+\sqrt{3s^2}$

4) $Pois(\lambda):\ \lambda\sim\bar{X}$

5) $Bin(n, p):\ n\sim\frac{\bar{X}^2}{\bar{X}-s^2},\ p\sim1+\bar{X}-\frac{\bar{X^2}}{\bar{X}}$

6) $Geom(p):\ p\sim\frac{1}{\bar{X}}$

7) $Beta(\alpha, \lambda):\ \alpha\sim\frac{(1-\bar{X})(\bar{X}-\bar{X^2})}{s^2},\ \lambda\sim\frac{\bar{X}-\bar{X^2}}{s^2}$


Оценки методом максимального правдоподобия:

1) $N(\mu, \sigma^2):\ \mu\sim\bar{X},\ \sigma^2\sim s^2$

2) $\Gamma(\alpha, \lambda):\ \alpha\sim\frac{\lambda}{\bar{X}}$

3) $R(a, b):\ a\sim\ X_{(1)},\ b\sim X_{(n)}$

4) $Pois(\lambda):\ \lambda\sim\bar{X}$

5) $Bin(n, p):\ p\sim\frac{\bar{X}}{n}$

6) $Geom(p):\ p\sim\frac{1}{\bar{X}}$

7) $Pareto(2, \theta),\ p(x)=\frac{2\theta^2}{x^3}I(x>\theta):\ \theta\sim X_{(1)}$

## Выборка размера $N = 10000$ из $Exp(\theta)$, где $\theta = 1$

In [ ]:
N = 10000
t = 1
exp_rv = sts.expon(t)
sample = exp_rv.rvs(N)

В массиве хранятся элементы, соответсвующие выборочному k-му моменту и оценке, для каждого $n\leq N$

$first \sim \bar{X^{k}}$

$result \sim \sqrt[k]{\frac{k!}{\bar{X^{k}} } }$ 

In [ ]:
def f(k, sample):
    avrg = float(sample[0])**k

    first = np.array([avrg])
    for x in xrange(1,10000):
        avrg = (avrg*x + sample[x]**k)/(x+1)   
        first = np.append(first, avrg)
        
    result = ((np.prod(k))/first)**(1.0/k)
    return result

#### Построение графиков модуля разности оценки и $\theta$

In [ ]:
x = np.linspace(0, N, N)
plt.plot(x, np.abs(f(1, sample) - t), label = '$k = 1$')
plt.plot(x, np.abs(f(2, sample) - t), label = '$k = 2$')
plt.plot(x, np.abs(f(5, sample) - t), label = '$k = 5$')
plt.plot(x, np.abs(f(10, sample) - t), label = '$k = 10$')
plt.plot(x, np.abs(f(20, sample) - t), label = '$k = 20$')
plt.plot(x, np.abs(f(50, sample) - t), label = '$k = 50$')
plt.plot(x, np.abs(f(100, sample) - t), label = '$k = 100$')
plt.plot(x, np.abs(f(200, sample) - t), label = '$k = 200$')
plt.plot(x, np.abs(f(300, sample) - t), label = '$k = 300$')
plt.plot(x, np.abs(f(400, sample) - t), label = '$k = 400$')
plt.plot(x, np.abs(f(500, sample) - t), label = '$k = 500$')
plt.ylabel('$|\hat{\\theta} - \\theta|$')
plt.xlabel('$n$')
plt.legend()
plt.show()

In [ ]:
x = np.linspace(0, N, N)
plt.plot(x, np.abs(f(1, sample) - t), label = '$k = 1$')
plt.plot(x, np.abs(f(2, sample) - t), label = '$k = 2$')
plt.plot(x, np.abs(f(3, sample) - t), label = '$k = 3$')
plt.plot(x, np.abs(f(4, sample) - t), label = '$k = 4$')
plt.plot(x, np.abs(f(5, sample) - t), label = '$k = 5$')
plt.plot(x, np.abs(f(10, sample) - t), label = '$k = 10$')
plt.plot(x, np.abs(f(20, sample) - t), label = '$k = 20$')
plt.ylabel('$|\hat{\\theta} - \\theta|$')
plt.xlabel('$n$')
plt.legend()
plt.show()

In [ ]:
x = np.linspace(0, N, N)
plt.plot(x, np.abs(f(20, sample) - t), label = '$k = 20$')
plt.plot(x, np.abs(f(50, sample) - t), label = '$k = 50$')
plt.plot(x, np.abs(f(100, sample) - t), label = '$k = 100$')
plt.plot(x, np.abs(f(200, sample) - t), label = '$k = 200$')
plt.plot(x, np.abs(f(300, sample) - t), label = '$k = 300$')
plt.plot(x, np.abs(f(400, sample) - t), label = '$k = 400$')
plt.plot(x, np.abs(f(500, sample) - t), label = '$k = 500$')
plt.ylabel('$|\hat{\\theta} - \\theta|$')
plt.xlabel('$n$')
plt.legend()
plt.show()

# Вывод

Лучше всего приближает значение параметра $\theta$ оценка при $k = 2$. С ростом k оценка ухудшается, но отклонение не превышает 1.